In [13]:
from rdkit import Chem
import torch
import sys
sys.path.append(os.getcwd())
from scaffold_constrained_model import scaffold_constrained_RNN 
from data_structs import Vocabulary, Experience
from scoring_functions import get_scoring_function
from utils import Variable, seq_to_smiles, fraction_valid_smiles, unique
from rdkit.Chem.Draw import IPythonConsole

voc = Vocabulary(init_from_file="data/DistributionLearningBenchmark/Voc")
    
Agent = scaffold_constrained_RNN(voc)
if torch.cuda.is_available():
    Agent.rnn.load_state_dict(torch.load('data/DistributionLearningBenchmark/Prior_sureChEMBL_randomized.ckpt'))
else:
    Agent.rnn.load_state_dict(torch.load('data/DistributionLearningBenchmark/Prior_sureChEMBL_randomized.ckpt', map_location=lambda storage, loc: storage))

# Sample multiple times

In [15]:
import matplotlib.pyplot as plt
from rdkit.Chem import Descriptors, Lipinski
from rdkit.Chem import QED
from rdkit.Chem import AllChem
from rdkit import DataStructs
from random import sample
import pandas as pd

def int_diversity(smiles):
    if len(smiles)==0:
        return 0.5
    diversity = 0
    for s in smiles:
        fp = AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(s), 4)
        for m in smiles:
            query_fp = AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(m), 4)
            diversity += DataStructs.TanimotoSimilarity(query_fp, fp) 
    diversity /= len(smiles)**2
    return 1 - diversity

def validity_and_uniqueness_assessement(pattern= "c1c(*)cc2c(c1)c(*)ccn2", refs = None, n_tryouts=5000, save_path=None):
    n_valid = 0
    n_unique = 0
    logP = []
    MW = []
    qed_list = []
    SAS = []
    HD_list = []
    HA_list = []
    smiles = []
    for i in range(n_tryouts):
        if i%1000 == 0:
            print("1000 more...")
        seqs, agent_likelihood, entropy, _ = Agent.sample(pattern)
        #seqs, agent_likelihood, entropy = Agent.sample(1)
        mol = Chem.MolFromSmiles(seq_to_smiles(seqs, voc)[0])
        if mol:
            n_valid += 1
            
            if Chem.MolToSmiles(mol) not in smiles:
                smiles.append(Chem.MolToSmiles(mol))
                n_unique += 1
            try:
                MW.append(Descriptors.ExactMolWt(mol))
                logP.append(Descriptors.MolLogP(mol))
                qed_list.append(QED.qed(mol))
                SAS.append(calculateSAS(mol))
                HA_list.append(Lipinski.NumHAcceptors(mol))
                HD_list.append(Lipinski.NumHDonors(mol))
            except:
                pass
    
    if save_path:
        
        df_summary = pd.DataFrame(columns = ["Scaffold", "Percentage valid molecules", "Percentage unique molecules"])
        df_summary["Scaffold"] = [pattern]
        df_summary["Percentage valid molecules"] = [n_valid/n_tryouts]
        df_summary["Percentage unique molecules"] = [n_unique/n_valid]

        df_details = pd.DataFrame(columns = ["Molecular weight",
                                    "Calculated log P", "Synthetizability Accessibility Score (SAS)", "Quantitative Estimate of Drug-likeness (QED)",
                                    "Number of H bonds acceptors", "Number of H bonds donors"])
        df_details["Molecular weight"] = MW
        df_details["Calculated log P"] = logP
        df_details["Synthetizability Accessibility Score (SAS)"] = SAS
        df_details["Quantitative Estimate of Drug-likeness (QED)"] = qed_list
        df_details["Number of H bonds acceptors"] = HA_list
        df_details["Number of H bonds donors"] = HD_list
        
        df_details.to_csv(save_path + "_details.csv")
        df_summary.to_csv(save_path + "_summary.csv")
    return n_valid/n_tryouts, n_unique/n_valid, MW, logP, SAS, qed_list, HA_list, HD_list

In [16]:
patterns = ["N(*)2C(=C)N(c3ccc(C#N)c(C(F)(F)F)c3)C(=O)C23CCC3",
            'C(*)N2CCN(C(=O)Oc3cncc(*)c3)C(C)C2',
            'c(*)1ccc(Nc2cnc(N)c(NCc3ccc4ncccc4c3)n2)nc1',
            'N(*)C(=O)C(=O)N2CCC(c3ccnc4ccc(*)cc34)CC2',
            'C(*)c1nc(=O)c(S(=O)(=O)c2ccc(*)c(*)c2)c(O)n1(*)',
            'C(*)c1cnn(CC2C(NC(=O)C(=NOC3(C(*)O)CC3)c3csc(*)n3)C(=O)N2(*))n1',
            'C(*)Oc1cc(*)c(*)c(c2ccc3nc(N(*))ncc3c2)c1(*)', 
            'c(*)1cnc2cc(*)c(*)cc2c1N1CCN(*)C(*)C1',
            'c1(*)cnc2cc(*)c(*)cc2c1N1CCNCC1',
            'C(*)NC2=NN=C(c3ccc4[nH]c(=O)[nH]c4c3)CS2', #
            'C(*)#Cc1ccc(OCc2nnc(S(*))n2-c2cccnc2)cc1(*)',
            'C(*)C1=C(*)C(*)N=C(c2nccs2)N1',
            'S(*)(C)(=O)CC(c1ccc(OC)cc1)n1c(=O)n(*)c2cc(-c3ccccc3(*))ccc21',
            'c(*)1ccc2[nH]c3c(c2c1)CN(c1ccc2oc(N4CCOCC4)nc2c1)CC3',
            'c(*)1cc(Nc2ncc(*)c(Nc3ccccc3(*))n2)c(O(*))cc1C(*)',
            'C=C(*)C(=O)CC(=O)C=C(*)',
            'C(*)N(CC2=C(C(=O)OCC)C(c3ccc(F)cc3(*))N=C(c3nccs3)N2)']

In [17]:
for i, pattern in enumerate(patterns):
    val, uniques, mw, logp, synthetic_as, qeds, _, _ = validity_and_uniqueness_assessement(pattern, n_tryouts=100)
    print(val)
    print(uniques)

1000 more...


/site/rp/work/projects/DD/Jupyter/notebooks/Maxime/scaffold-constrained-generation/scaffold_constrained_model.py:167: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = F.softmax(logits)
/site/rp/work/projects/DD/Jupyter/notebooks/Maxime/scaffold-constrained-generation/scaffold_constrained_model.py:168: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  log_prob = F.log_softmax(logits)
/site/rp/work/projects/DD/Jupyter/notebooks/Maxime/scaffold-constrained-generation/scaffold_constrained_model.py:379: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = F.softmax(logits)
/site/rp/work/projects/DD/Jupyter/notebooks/Maxime/scaffold-constrained-generation/scaffold_constrained_model.py:380: UserWarning: Implicit dimension choice for log_softmax has been deprecat

0.94
0.9361702127659575
1000 more...
0.88
0.9090909090909091
1000 more...
0.78
0.6666666666666666
1000 more...
0.89
0.9550561797752809
1000 more...
0.93
0.956989247311828
1000 more...
0.94
0.9787234042553191
1000 more...
0.89
1.0
1000 more...
0.74
1.0
1000 more...
0.8
0.9625
1000 more...
0.93
0.7634408602150538
1000 more...
0.76
0.9868421052631579
1000 more...
0.92
0.9565217391304348
1000 more...
0.93
1.0
1000 more...
0.73
0.6301369863013698
1000 more...
0.76
0.9868421052631579
1000 more...
0.91
0.8681318681318682
1000 more...
0.92
0.9239130434782609
